In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
from dash_table.Format import Format, Scheme, Group, Symbol

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "aacpassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
                               
dfwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
                               
dfmwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Huskey","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
                               
dfdit = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Brandon Kelfstrom'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
             children = [
                 dcc.RadioItems(
                 id='filter-type',
                 options=[
                    {'label': 'Water Rescue', 'value': 'WR'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                    {'label': 'Reset', 'value':'R'}
                 ],
                 value='R',
                 labelStyle={'display': 'inline-block'}
                 ),
                 html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={
                        'height' : '50%',
                        'width' : '50%',
                        'float' : 'right',
                        'position' : 'relative',
                        'padding-top' : 0,
                        'padding-right' : 0
                 })]),
        
             ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}
        ],
    data=df.to_dict('records'),
#Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### Code to filter interactive data table with MongoDB queries
        if filter_type == 'R':
            new_df = df
        elif filter_type == 'WR':
            new_df = dfwr
        elif filter_type == 'MWR':
            new_df = dfmwr
        elif filter_type == 'DIT':
            new_df = dfdit
        else:
            new_df = df
            
        data = new_df.to_dict('records')
        
        columns=[
            {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}  
        ]
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # add code for chart of your choice (e.g. pie chart) #
    df2 = pd.DataFrame.from_dict(viewData)
    newdata = df2.breed.value_counts()
    newdata.name = "breed"
    newdata = newdata.to_frame()
    newData.reset_index(inplace = True)
                             
    return [
        dcc.Graph(            
            figure = new_data, values = "breed", names = "index"
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#Code for your geolocation chart
    odff = df if viewData is None else pd.DataFrame.from_dict(viewData)
    if row is None:
        dff = odff
    else:
        dff = odff.iloc[row,:]

        map = [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30,-100], zoom=10, children = [
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0,9])
                    ])
                ])
            ])
        ]
        return map


app